## Cluster Analysis with Multinomial EM (Text Clustering)

#### In the K-means clustering algorithm we forced documents to be assigned to one cluster. This is known as hard clustering. This approach in not really accurate when working with language.  When someone writes about a topic they use a certain terminology to describe that topic. We assume that when someone else writes about the same topic they are likely to use the same terminology or words. But words may also be used across multiple topics and a clinical note may describe multiple topics. It may be more appropriate to assign words and documents to multiple topics (or clusters) with a certain probability based on their use.

#### This example illustrates how to group a set of different clinical notes based on their topics and the terminology used to describe the topics. We will model our clinical notes as a collection of unigram and bigram words. Specifically, we will represent each clinical note as a feature set of unigram and bigram frequencies found in the clinical note. We will use a matrix where each row will represent a clinical note and each column a feature (i.e. distinct unigram or bigram).

### Mixture of Multinomial Distributions

#### Text is best represented as a mixture of multinomial distributions where each topic has a particular multinomial distribution associated with it and each document in a mixture of topics. 

#### Formally, let $p(c)\space=\space\pi_c$ be the prior probability of a document containing topic c, and each topic c is represented as a multinomial distribution $p(D_i|c)$ with parameters $\mu_{jc}$, then each document becomes a mixture over topics as

#### $p(D_i)\space=\space \displaystyle\sum_{c=1}^{n_c} p(D_i|c)p(c) = \displaystyle\sum_{c=1}^{n_c} \pi_c \prod_{j=1}^{n_w}\mu_{jc}^{T_{ij}}$

### Expectation Maximization for Mixtures of Multinomials

#### The expectation maximization algorithm will allow us to fit a multinomial mixture model to our data. Our goal is to identify which documents belong to which topics and what words (unigrams and bigrams) are used to describe the topic. 

#### 1. E-Step. Compute the expectation that document $D_i$ belongs to topic (cluster) $c$

####         $\gamma_{ic} \propto \pi_c \displaystyle\prod_{j = 1}^{n_w} \mu_{jc}^{T_{ij}}$

#### $Note: \space normalize \space expectations \space over \space c$

#### where,
#### $\pi_c \equiv prior{\space}probability{\space}of{\space}document{\space}containing{\space}topic{\space}c$
#### $\mu_{jc} \equiv probability{\space}of{\space}w_j{\space}in{\space}topic{\space}c$
#### $T_{ij} \equiv count \space of \space w_j \space in \space topic \space c$


#### 2. M-Step. Update the mixture parameters. 

#### $\mu_{jc} = \frac{\sum_{i = 1}^{n_d} \gamma_{ic}T_{ij}}{\sum_{i = 1}^{n_d}\sum_{l = 1}^{n_w} \gamma_{ic}T_{il}} \equiv probability \space of \space a \space word \space being \space w_j \space in \space topic \space (cluster) \space c$ 

#### $\pi_c = \frac{1}{n_d} \displaystyle \sum_{i = 1}^{n_d} \gamma_{ic} \equiv prior \space probability \space of \space each \space cluster$

#### $Note: \space normalize \space priors \space uniformly,\space initialize \space \mu's \space to \space multinomial \space generated \space from \space uniform \space dirichlet \space distribution \space such \space that \sum_{j = 1}^{n_w} \mu_{jc} = 1$

### Implementation

#### Our implementation will consist of miipulating 4 matricies as described below.

![title](./images/MultinomialEMClustering.png)


### Environment Setup

#### First we will import some Python packages that we will use.

In [1]:
import nltk
import re
import pandas as pd
import numpy as np 
import numpy.matlib as ml
import pickle as pkl
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

### Syntactic NLP Processing

#### We need a function to tokenize our text and remove noise like dates, ages, etc.

In [2]:
def tokenize(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = [ token for token in tokens if re.search('(^[a-zA-Z]+$)', token) ]
    return filtered_tokens

### Retrieving our Corpus

#### Let's pull in our corpus that we had serialized out to disk.  

In [3]:
file = open('differential-corpus.pkl','rb')
corpus = pkl.load(file)
file.close()

### More Syntactic Processing

#### We will want to get ride of stop words that are essentially noise. 

In [4]:
cachedStopWords = stopwords.words("english") + ['year', 'old', 'man', 'woman', 'ap', 'am', 'pm', 'portable', 'pa', 'lat', 'admitting', 'diagnosis', 'lateral']
print(cachedStopWords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Generate Document-Term Frequency Counts

#### In this step we tokenize our text and remove stop words in addition to generating our frequency counts.

#### 1) how many documents are we working with and how many features (unigrams & bigrams)?

#### 2) Can you figure out what max_df and min_df is doing to our feature count?

In [5]:
corpusList = list(corpus.values())
labels = list(corpus.keys())
rank = list(range(len(labels)))

cv = CountVectorizer(lowercase=True, max_df=0.80, max_features=None, min_df=0.033,
                     ngram_range=(2, 2), preprocessor=None, stop_words=cachedStopWords,
                     strip_accents=None, tokenizer=tokenize, vocabulary=None)
SparseT = cv.fit_transform(corpusList)
print("The dimensions of our document-term matrix")
print(SparseT.shape)
print()

The dimensions of our document-term matrix
(1500, 291)



### Feature Set

#### Let's take a look at our feature set.

#### 1) Do we have a lot of noise in our features?

In [6]:
lexicon = cv.get_feature_names()
print (lexicon)
print()

['abdomen contrast', 'abdomen pelvis', 'acute sob', 'adrenal glands', 'air space', 'also noted', 'amt final', 'amt underlying', 'aortic stenosis', 'approximately cm', 'atelectasis left', 'axial images', 'bilateral pleural', 'bilateral pulmonary', 'bone windows', 'cad chf', 'cardiac mediastinal', 'cardiac silhouette', 'catheter terminates', 'catheter tip', 'cc optiray', 'central line', 'central venous', 'change final', 'chest clip', 'chest compared', 'chest comparison', 'chest contrast', 'chest ct', 'chest different', 'chest history', 'chest indication', 'chest obtained', 'chest pain', 'chest physician', 'chest port', 'chest radiograph', 'chest single', 'chest w', 'chf final', 'chf reason', 'chf underlying', 'clinical indication', 'cm carina', 'compared previous', 'compared prior', 'comparison chest', 'comparison findings', 'comparison made', 'condition acute', 'condition cad', 'condition chf', 'condition copd', 'condition critical', 'condition reason', 'condition sob', 'condition yo', 

## Define EM Algorithm (E-Step & M-Step)

#### Now let's define our EM Algorithm.

#### 1) In the ExpD function why are we multiplying by $1x10^2 \space ?$  

In [7]:
  
def ExpD(T, Mu, Pi):    
    C_n = Pi.shape[1]
    D_n = T.shape[0]
    Gamma = ml.zeros((D_n, C_n))
    Gamma.astype('float64')
    for c in range(0, C_n):
        Gamma[:, c] = Pi[0][c] * ((Mu[:,c].A[:,0]*1e2)**T).prod(1)
    Gamma = Gamma / Gamma.sum(axis=1)
    return Gamma
    
def updateMu(T, Gamma):    
    C_n = Gamma.shape[1]
    W_n = T.shape[1]
    Mu = ml.zeros((W_n, C_n))
    for c in range(0, C_n):
        numerator = sum(np.multiply(Gamma[:,c],T)).T
        demoninator = sum(np.multiply(Gamma[:,c],T).sum(1))
        Mu[:,c] = numerator / demoninator
    return Mu
    
def updatePi(Gamma):    
    D_n = Gamma.shape[0]
    Pi = sum(Gamma) / D_n
    return Pi.A
    

## Let's Run it !

#### Let's start with 10 topics (clusters) and we will interate 100 times. EM converges quickly.

#### 1) Can you determine at what iteration we are starting to reach convergence?

In [8]:
T = SparseT.todense()
D_n, W_n = T.shape
C_n = 10
Pi = ml.repmat(1/C_n, 1, C_n)
Mu = ml.mat(np.random.dirichlet(np.ones(W_n), C_n).T)
for i in range(1,101):
    print('Iteration: ' + str(i)) 
    Gamma = ExpD(T, Mu, Pi)
    print(Gamma.sum(0))
    Mu = updateMu(T, Gamma)
    Pi = updatePi(Gamma)


Iteration: 1
[[193.30200411 135.11259119 141.80114131 113.31805442  69.03015579
  284.62954206 159.59297205  66.52396698 180.54655495 156.14301715]]
Iteration: 2
[[188.88501371 213.35467053 150.09219043 175.3090981   80.126764
  249.64890222 163.8370703   24.33589956 183.48144142  70.92894973]]
Iteration: 3
[[181.65751009 217.21096588 151.88083951 160.93569804 101.75842349
  239.76115061 178.29536587  23.27442083 190.75823345  54.46739223]]
Iteration: 4
[[172.19529914 212.80730333 145.99154887 163.30447196 114.72007021
  236.13136069 186.1061887   23.60665037 186.98542353  58.15168321]]
Iteration: 5
[[167.5334502  215.49416731 143.7677047  161.45132168 119.54875758
  231.30313559 192.13504349  23.62989261 180.83974101  64.29678582]]
Iteration: 6
[[166.26770981 215.94738992 142.97852736 157.10443774 120.31033271
  225.84191842 202.65406207  23.60171052 180.05530737  65.23860408]]
Iteration: 7
[[162.98613842 215.3101063  140.11654822 154.11653049 121.06829242
  226.17844864 211.40179363 

Iteration: 56
[[148.458191   240.9076558  126.26070048 138.58979699 111.40753871
  223.52514405 241.65132005  23.6048376  176.07055722  69.52425811]]
Iteration: 57
[[148.45407746 240.97031975 126.15231218 138.63902795 111.38795984
  223.46630736 241.73360829  23.60525525 176.06974194  69.52138999]]
Iteration: 58
[[148.45058169 241.0317758  126.05056936 138.69298858 111.36402447
  223.39121415 241.82647946  23.60566707 176.06881131  69.51788812]]
Iteration: 59
[[148.44756905 241.09622731 125.95589009 138.75385639 111.32932262
  223.29569676 241.93368307  23.60608625 176.06778654  69.51388192]]
Iteration: 60
[[148.44485177 241.16523578 125.86792639 138.82271952 111.27490125
  223.17889847 242.06278291  23.60651204 176.06668009  69.50949178]]
Iteration: 61
[[148.44215952 241.2399677  125.78572509 138.89934307 111.18322429
  223.04100248 242.23129156  23.60692786 176.06550662  69.50485181]]
Iteration: 62
[[148.4390092  241.32207902 125.70783512 138.98181283 111.01442152
  222.88146222 242.

## Examination of Clusters and Terminology

#### Let's take a look at the top cluster for each clinical note and the top 20 words to distinguish this topic.

#### 1) Is there noise in the terminology? If there is how can we get ride of it?


In [11]:
clusters = Gamma.argmax(1).A
clusters = [i[0] for i in clusters]
clinicalDocuments = { 'labels': labels, 'rank': rank, 'corpus': corpusList, 'cluster': clusters }
frame = pd.DataFrame(clinicalDocuments, index = clusters , columns = ['rank', 'labels', 'corpus', 'cluster'])
grouped = frame['rank'].groupby(frame['cluster'])
topWords = Mu.T.argsort()[:, ::-1]
for i in range(C_n):
    n = len(frame.loc[i]['labels'].values.tolist())
    print('Cluster %d (%d):,' % (i+1, n), end='')
    for label in frame.ix[i]['labels'].values.tolist():
        print(' %s,' % label, end='')
    print()
    print()
    print('           Words:', end='')
    for indice in list(topWords[i, :20].A[0]):
        print(' %s (%.5f)' % (lexicon[indice], Mu[indice,i]), end=',')
    print()
    print()
    print()

Cluster 1 (147):, PNA48, PNA97, PNA98, PNA103, PNA186, PNA187, PNA200, PNA201, PNA214, PNA215, PNA262, PNA278, PNA279, PNA280, PNA281, PNA313, PNA386, PNA387, PNA391, PNA392, PNA445, PNA446, PNA497, CHF53, CHF54, CHF65, CHF79, CHF123, CHF125, CHF151, CHF152, CHF160, CHF161, CHF201, CHF210, CHF211, CHF215, CHF231, CHF233, CHF234, CHF235, CHF236, CHF278, CHF285, CHF297, CHF300, CHF309, CHF310, CHF324, CHF325, CHF326, CHF327, CHF328, CHF330, CHF331, CHF334, CHF335, CHF367, CHF368, CHF378, CHF388, CHF391, CHF398, CHF403, CHF404, CHF412, CHF415, CHF419, CHF420, CHF421, CHF422, CHF423, CHF439, CHF440, CHF443, CHF444, CHF452, CHF453, CHF460, CHF461, CHF481, CHF482, CHF484, CHF494, CHF495, COPD53, COPD54, COPD65, COPD79, COPD123, COPD125, COPD151, COPD152, COPD160, COPD161, COPD201, COPD210, COPD211, COPD215, COPD231, COPD233, COPD234, COPD235, COPD236, COPD278, COPD285, COPD297, COPD300, COPD309, COPD310, COPD324, COPD325, COPD326, COPD327, COPD328, COPD330, COPD331, COPD334, COPD335, COPD367

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


 PNA31, PNA37, PNA67, PNA94, PNA95, PNA102, PNA106, PNA116, PNA127, PNA128, PNA139, PNA152, PNA157, PNA160, PNA168, PNA170, PNA179, PNA180, PNA182, PNA189, PNA190, PNA191, PNA192, PNA197, PNA199, PNA202, PNA203, PNA204, PNA216, PNA219, PNA223, PNA224, PNA226, PNA227, PNA235, PNA237, PNA238, PNA239, PNA241, PNA253, PNA255, PNA264, PNA266, PNA269, PNA270, PNA272, PNA283, PNA284, PNA290, PNA296, PNA303, PNA304, PNA305, PNA310, PNA311, PNA312, PNA325, PNA330, PNA333, PNA351, PNA370, PNA372, PNA376, PNA377, PNA379, PNA380, PNA381, PNA383, PNA390, PNA398, PNA401, PNA406, PNA408, PNA415, PNA418, PNA420, PNA425, PNA430, PNA434, PNA443, PNA447, PNA448, PNA449, PNA456, PNA465, PNA467, PNA468, PNA477, PNA479, PNA482, PNA483, PNA491, PNA493, CHF8, CHF24, CHF39, CHF43, CHF86, CHF88, CHF96, CHF106, CHF145, CHF155, CHF157, CHF168, CHF169, CHF170, CHF171, CHF173, CHF179, CHF180, CHF183, CHF189, CHF202, CHF226, CHF246, CHF250, CHF251, CHF252, CHF253, CHF254, CHF257, CHF258, CHF261, CHF266, CHF272, CHF2

### Soft Clustering Document Examination

#### 1) Can you find clinical notes that belong to more than 1 cluster ?


In [12]:
N, C = Gamma.shape
for i in range(N):
    print('%s: ' % labels[i], end='')
    for j in range(C):
        print('C%d (%.7f): ' % (j+1, Gamma[i, j]), end='')
    print()
    print()
print()

PNA1: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (1.0000000): 

PNA2: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (1.0000000): C10 (0.0000000): 

PNA3: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (1.0000000): 

PNA4: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (1.0000000): C10 (0.0000000): 

PNA5: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (1.0000000): C10 (0.0000000): 

PNA6: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (1.0000000): C10 (

PNA64: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (1.0000000): 

PNA65: C1 (0.0000000): C2 (0.0000303): C3 (0.0000000): C4 (0.9983776): C5 (0.0000000): C6 (0.0000015): C7 (0.0015905): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA66: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0022394): C7 (0.9977606): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA67: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.9999858): C7 (0.0000142): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA68: C1 (0.0000000): C2 (1.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA69: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0002848):

PNA141: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0044695): C10 (0.9955305): 

PNA142: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (1.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA143: C1 (0.0000000): C2 (0.9713338): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0286662): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA144: C1 (0.0000000): C2 (0.9981694): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0013535): C7 (0.0004772): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA145: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (1.0000000): C10 (0.0000000): 

PNA146: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (1.000

PNA208: C1 (0.0000000): C2 (0.0008454): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.9991546): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA209: C1 (0.0000000): C2 (1.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA210: C1 (0.0000000): C2 (0.9999779): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000221): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA211: C1 (0.0000000): C2 (1.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA212: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (1.0000000): C10 (0.0000000): 

PNA213: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (1.000

PNA264: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA265: C1 (0.0000154): C2 (0.0447980): C3 (0.0000000): C4 (0.8814316): C5 (0.0000000): C6 (0.0001182): C7 (0.0736368): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA266: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA267: C1 (0.0000000): C2 (0.1007087): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0390347): C7 (0.8602566): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA268: C1 (0.0000017): C2 (0.9999983): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA269: C1 (0.0000000): C2 (0.0001767): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.9998233): C7 (0.0000000): C8 (0.0000000): C9 (0.000

...

COPD138: C1 (0.0000000): C2 (1.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD139: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (1.0000000): C10 (0.0000000): 

COPD140: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (1.0000000): C10 (0.0000000): 

COPD141: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD142: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD143: C1 (0.0000000): C2 (1.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 

COPD222: C1 (0.0000000): C2 (0.0029310): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.9970690): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD223: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (1.0000000): C10 (0.0000000): 

COPD224: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (1.0000000): C10 (0.0000000): 

COPD225: C1 (0.0000000): C2 (1.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD226: C1 (0.0000000): C2 (0.0003822): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.9992530): C7 (0.0003648): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD227: C1 (0.0000000): C2 (0.0001363): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.9998637): C8 (0.0000000): C9 

COPD305: C1 (0.0000000): C2 (0.0000001): C3 (0.0000000): C4 (0.9999999): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD306: C1 (0.0000000): C2 (0.0000001): C3 (0.0000000): C4 (0.9999999): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD307: C1 (0.0000000): C2 (0.0001790): C3 (0.0000000): C4 (0.0000158): C5 (0.0000000): C6 (0.0000000): C7 (0.9998052): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD308: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (1.0000000): 

COPD309: C1 (0.9999999): C2 (0.0000000): C3 (0.0000000): C4 (0.0000001): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD310: C1 (0.9999999): C2 (0.0000000): C3 (0.0000000): C4 (0.0000001): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 

COPD388: C1 (0.9999973): C2 (0.0000027): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD389: C1 (0.0000000): C2 (0.0000000): C3 (0.0000023): C4 (0.0000000): C5 (0.9999976): C6 (0.0000001): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD390: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.9959942): C6 (0.0025803): C7 (0.0014255): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD391: C1 (1.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD392: C1 (0.0000000): C2 (0.0000047): C3 (0.9999951): C4 (0.0000000): C5 (0.0000000): C6 (0.0000002): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD393: C1 (0.0000000): C2 (0.0000001): C3 (0.9999999): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 

COPD472: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD473: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD474: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD475: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD476: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

COPD477: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 

### Soft Clustering Term Examination

#### 1) Can you find terms that belong to more than 1 cluster ?

In [13]:
W, C = Mu.shape
for i in range(W):
    print('%s: ' % lexicon[i], end='')
    for j in range(C):
        print('C%d (%.5f): ' % (j+1, Mu[i, j]), end='')
    print()
    print()
print()

abdomen contrast: C1 (0.00000): C2 (0.00000): C3 (0.00000): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.00132): C9 (0.00703): C10 (0.01283): 

abdomen pelvis: C1 (0.00000): C2 (0.00000): C3 (0.00000): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.00132): C9 (0.00591): C10 (0.00855): 

acute sob: C1 (0.00000): C2 (0.00111): C3 (0.01180): C4 (0.00082): C5 (0.00000): C6 (0.00531): C7 (0.00045): C8 (0.00000): C9 (0.00102): C10 (0.00000): 

adrenal glands: C1 (0.00000): C2 (0.00000): C3 (0.00000): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.00132): C9 (0.00570): C10 (0.00764): 

air space: C1 (0.00127): C2 (0.00164): C3 (0.00000): C4 (0.00164): C5 (0.00131): C6 (0.00806): C7 (0.00113): C8 (0.00265): C9 (0.00112): C10 (0.00000): 

also noted: C1 (0.00000): C2 (0.00180): C3 (0.00000): C4 (0.00000): C5 (0.00044): C6 (0.00129): C7 (0.00222): C8 (0.00596): C9 (0.00153): C10 (0.00275): 

amt final: C1 (0.00000): C2 (0.00000): C3 (0.00000): C4 (0

condition critical: C1 (0.00646): C2 (0.00216): C3 (0.00224): C4 (0.00165): C5 (0.00000): C6 (0.00000): C7 (0.00225): C8 (0.00000): C9 (0.00000): C10 (0.00000): 

condition reason: C1 (0.00000): C2 (0.00150): C3 (0.00164): C4 (0.00544): C5 (0.00000): C6 (0.00087): C7 (0.00191): C8 (0.00066): C9 (0.00143): C10 (0.00061): 

condition sob: C1 (0.00458): C2 (0.00216): C3 (0.00000): C4 (0.00000): C5 (0.00087): C6 (0.00153): C7 (0.00376): C8 (0.00000): C9 (0.00000): C10 (0.00000): 

condition yo: C1 (0.00407): C2 (0.00036): C3 (0.00207): C4 (0.00493): C5 (0.01314): C6 (0.00022): C7 (0.00112): C8 (0.00000): C9 (0.00041): C10 (0.00000): 

congestive heart: C1 (0.02485): C2 (0.00568): C3 (0.00906): C4 (0.01708): C5 (0.04202): C6 (0.00150): C7 (0.02627): C8 (0.01059): C9 (0.00102): C10 (0.00734): 

consolidation left: C1 (0.00000): C2 (0.00465): C3 (0.00209): C4 (0.00041): C5 (0.00000): C6 (0.00085): C7 (0.00136): C8 (0.00000): C9 (0.00132): C10 (0.00000): 

consolidation right: C1 (0.00102): C2

left lung: C1 (0.00052): C2 (0.00970): C3 (0.00547): C4 (0.00574): C5 (0.00601): C6 (0.00046): C7 (0.00093): C8 (0.00265): C9 (0.00244): C10 (0.00336): 

left pleural: C1 (0.01205): C2 (0.01831): C3 (0.00283): C4 (0.01780): C5 (0.00789): C6 (0.00063): C7 (0.00308): C8 (0.00926): C9 (0.00224): C10 (0.00336): 

left retrocardiac: C1 (0.00153): C2 (0.00205): C3 (0.00488): C4 (0.00082): C5 (0.00438): C6 (0.00544): C7 (0.00090): C8 (0.00000): C9 (0.00000): C10 (0.00000): 

left side: C1 (0.00458): C2 (0.00062): C3 (0.00000): C4 (0.00164): C5 (0.00178): C6 (0.00000): C7 (0.00134): C8 (0.00132): C9 (0.00154): C10 (0.00072): 

left sided: C1 (0.00000): C2 (0.00429): C3 (0.00244): C4 (0.00499): C5 (0.00219): C6 (0.00000): C7 (0.00244): C8 (0.00265): C9 (0.00051): C10 (0.00306): 

left subclavian: C1 (0.00889): C2 (0.00229): C3 (0.00733): C4 (0.00986): C5 (0.00438): C6 (0.00150): C7 (0.00092): C8 (0.00000): C9 (0.00143): C10 (0.00031): 

left upper: C1 (0.00305): C2 (0.01022): C3 (0.00000): C4 (

report history: C1 (0.00457): C2 (0.00626): C3 (0.01101): C4 (0.00289): C5 (0.00865): C6 (0.00688): C7 (0.00725): C8 (0.00288): C9 (0.00433): C10 (0.00220): 

report indication: C1 (0.01163): C2 (0.01669): C3 (0.02429): C4 (0.01926): C5 (0.01951): C6 (0.02147): C7 (0.02903): C8 (0.01141): C9 (0.01132): C10 (0.01170): 

report indications: C1 (0.00153): C2 (0.00135): C3 (0.00000): C4 (0.00314): C5 (0.00481): C6 (0.00571): C7 (0.00091): C8 (0.00000): C9 (0.00031): C10 (0.00061): 

resp distress: C1 (0.00000): C2 (0.00089): C3 (0.00000): C4 (0.00158): C5 (0.00088): C6 (0.00749): C7 (0.00000): C8 (0.00000): C9 (0.00061): C10 (0.00000): 

respiratory distress: C1 (0.00000): C2 (0.00409): C3 (0.00505): C4 (0.00917): C5 (0.00000): C6 (0.00369): C7 (0.00132): C8 (0.00000): C9 (0.00195): C10 (0.00000): 

respiratory failure: C1 (0.00102): C2 (0.00466): C3 (0.00000): C4 (0.00081): C5 (0.00000): C6 (0.00629): C7 (0.00548): C8 (0.00199): C9 (0.00112): C10 (0.00306): 

retrocardiac opacity: C1 (0.0